In [1]:
import wk_db

wk_db.read_sql('select count(*) from us_market_daily', db_loc='low_freq_db')

,count(*)
0,32616023


In [2]:
idx_map = {
    # 'spx': "SPX.GI",
    # 'ndx': "NDX.GI"
    # # "000001.SH":"上证综指",
    # "000016.SH":"上证50",
    'csi300': "000300.SH" # :"沪深300",
    # "399905.SZ":"中证500",
    # "000852.SH":"中证1000",
    # "399006.SZ":"创业板指"
}

In [3]:

idx_df = wk_db.read_sql(f"SELECT * FROM index_components_weight;", db_loc='low_freq_db')
idx_df

,trade_dt,index_windcode,con_windcode,industry_name,weight
0,2010-01-11,000016.SH,600000.SH,银行(中信),4.110
1,2010-01-11,000016.SH,600005.SH,钢铁(中信),0.920
2,2010-01-11,000016.SH,600015.SH,银行(中信),1.350
3,2010-01-11,000016.SH,600016.SH,银行(中信),5.550
4,2010-01-11,000016.SH,600018.SH,交通运输(中信),0.890
...,...,...,...,...,...
877301,2024-10-31,000852.SH,001270.SZ,国防军工(中信),0.071
877302,2024-10-31,000852.SH,001287.SZ,电子(中信),0.072
877303,2024-10-31,000852.SH,001301.SZ,基础化工(中信),0.133
877304,2024-10-31,000852.SH,001308.SZ,家电(中信),0.027


In [4]:
idx_name = 'csi300'
idx_code = idx_map[idx_name]

idx_df = wk_db.read_sql(f"SELECT con_windcode, MIN(trade_dt) as st, MAX(trade_dt) as ed FROM index_components_weight WHERE index_windcode='{idx_code}' GROUP BY con_windcode ORDER BY con_windcode;", db_loc='low_freq_db')
idx_df

,con_windcode,st,ed
0,000001.SZ,2010-01-04,2024-12-02
1,000002.SZ,2010-01-04,2024-12-02
2,000008.SZ,2016-12-30,2018-06-01
3,000009.SZ,2010-01-04,2017-12-01
4,000012.SZ,2010-01-04,2014-12-01
...,...,...,...
768,688363.SH,2021-06-30,2024-12-02
769,688396.SH,2021-06-30,2024-12-02
770,688561.SH,2021-12-31,2024-06-03
771,688599.SH,2021-12-31,2024-12-02


In [5]:
import pandas as pd

filter_date = pd.to_datetime('2024-01-01')
idx_df = idx_df[idx_df['ed'] > filter_date]
idx_df['st'] = '1900-01-01'
idx_df['ed'] = '2099-01-01'
idx_df['con_windcode'] = idx_df['con_windcode'].apply(lambda s: s.split('.')[1] +  s.split('.')[0])
idx_df

/tmp/ipykernel_2051046/2984568573.py:4: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  idx_df = idx_df[idx_df['ed'] > filter_date]
/tmp/ipykernel_2051046/2984568573.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idx_df['st'] = '1900-01-01'
/tmp/ipykernel_2051046/2984568573.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,con_windcode,st,ed
0,SZ000001,1900-01-01,2099-01-01
1,SZ000002,1900-01-01,2099-01-01
14,SZ000063,1900-01-01,2099-01-01
16,SZ000069,1900-01-01,2099-01-01
18,SZ000100,1900-01-01,2099-01-01
...,...,...,...
768,SH688363,1900-01-01,2099-01-01
769,SH688396,1900-01-01,2099-01-01
770,SH688561,1900-01-01,2099-01-01
771,SH688599,1900-01-01,2099-01-01


In [6]:
idx_df.to_csv(f'/data/linq/.qlib/qlib_data/cn_data/instruments/{idx_name}_ext.txt', index=False, header=False, sep='\t')